In [1]:
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np
import pandas as pd

def load_from_tsfile(file_path, return_y=True):
    X, y = load_from_tsfile_to_dataframe(file_path)
    X = pd.DataFrame({i: pd.Series(x) for i, x in enumerate(X.iloc[:, 0])})
    if return_y:
        return X, y
    else:
        return X


# Test the function
dname= 'SelfRegulationSCP1'
file_path = f'./datasets/{dname}/{dname}_TRAIN.ts'
X, y = load_from_tsfile(file_path)
print(X.shape, y.shape)



C:\Users\ui572274\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


(896, 268) (268,)


In [2]:
file_path = f'./datasets/{dname}/{dname}_TEST.ts'
X_test, y_test = load_from_tsfile(file_path)
print(X_test.shape, y_test.shape)

(896, 293) (293,)


In [3]:
X_train  = X.transpose()

In [4]:
X_test = X_test.transpose()

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
y = label_encoder.fit_transform(y)
y_test = label_encoder.fit_transform(y_test)


In [6]:
num_classes = len(np.unique(y_test))

num_classes

2

In [7]:
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score

def objective(trial):
    param = {
        'objective': 'binary',  # or 'multiclass' for multi-class classification
        'metric': 'binary_logloss',  # or 'multi_logloss' for multi-class classification

        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
    }

    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy



In [8]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



[I 2024-02-08 17:40:34,234] A new study created in memory with name: no-name-4394b376-623f-48fc-b743-16373328f1bf
[I 2024-02-08 17:40:35,540] Trial 0 finished with value: 0.7815699658703071 and parameters: {'num_leaves': 126, 'learning_rate': 5.588813005456218e-05, 'n_estimators': 543}. Best is trial 0 with value: 0.7815699658703071.
[I 2024-02-08 17:40:36,149] Trial 1 finished with value: 0.8395904436860068 and parameters: {'num_leaves': 47, 'learning_rate': 0.22904633003128042, 'n_estimators': 500}. Best is trial 1 with value: 0.8395904436860068.
[I 2024-02-08 17:40:36,736] Trial 2 finished with value: 0.5017064846416383 and parameters: {'num_leaves': 7, 'learning_rate': 3.9734908703337973e-07, 'n_estimators': 389}. Best is trial 1 with value: 0.8395904436860068.
[I 2024-02-08 17:40:37,888] Trial 3 finished with value: 0.8156996587030717 and parameters: {'num_leaves': 215, 'learning_rate': 2.9108308094897447e-05, 'n_estimators': 511}. Best is trial 1 with value: 0.8395904436860068.
[

Number of finished trials: 100
Best trial: {'num_leaves': 191, 'learning_rate': 0.4117942969579083, 'n_estimators': 801}
